In [1]:
import pandas as pd
import geopandas as gpd
import os
import fiona

## TAZ base-year data

In [6]:
# TAZ geography
url = 'https://opendata.arcgis.com/datasets/94e6e7107f0745b5b2aabd651340b739_0.geojson'
taz = gpd.read_file(url)
display(taz.head())

,objectid,taz1454,district,county,gacres,Shape__Area,Shape__Length,geometry
0,1,566,11,Santa Clara,1708.683056,0.000444,0.097550,"POLYGON ((-121.86806 37.28720, -121.86775 37.2..."
1,2,565,11,Santa Clara,486.200669,0.000126,0.054171,"POLYGON ((-121.85877 37.28024, -121.85903 37.2..."
2,3,573,11,Santa Clara,538.069933,0.000140,0.057053,"POLYGON ((-121.81238 37.28889, -121.81228 37.2..."
3,4,571,11,Santa Clara,450.407906,0.000117,0.052042,"POLYGON ((-121.80597 37.27730, -121.80558 37.2..."
4,5,576,11,Santa Clara,423.273023,0.000110,0.043438,"POLYGON ((-121.82788 37.30855, -121.82648 37.3..."


In [8]:
# base-year attributes
# file location: 'https://github.com/BayAreaMetro/petrale/blob/master/applications/travel_model_lu_inputs/2015/TAZ1454%202015%20Land%20Use.csv'
att_raw = pd.read_csv(r'C:\Users\ywang\Documents\GitHub\petrale\applications\travel_model_lu_inputs\2015\TAZ1454 2015 Land Use.csv')
att = att_raw.loc[:,['ZONE','DISTRICT','SD','COUNTY','TOTHH','TOTPOP','TOTEMP','SFDU','MFDU']]
att['UNITS'] = att['SFDU'] + att['MFDU']
taz_att = taz.merge(att, left_on = 'objectid', right_on = 'ZONE', how = 'left')
display(taz_att.head())
taz_att.to_file('MapForShare/taz_att.shp')

,objectid,taz1454,district,county,gacres,Shape__Area,Shape__Length,geometry,ZONE,DISTRICT,SD,COUNTY,TOTHH,TOTPOP,TOTEMP,SFDU,MFDU,UNITS
0,1,566,11,Santa Clara,1708.683056,0.000444,0.097550,"POLYGON ((-121.86806 37.28720, -121.86775 37.2...",1,1,1,1,34,64,16674,0,39,39
1,2,565,11,Santa Clara,486.200669,0.000126,0.054171,"POLYGON ((-121.85877 37.28024, -121.85903 37.2...",2,1,1,1,170,320,25943,0,198,198
2,3,573,11,Santa Clara,538.069933,0.000140,0.057053,"POLYGON ((-121.81238 37.28889, -121.81228 37.2...",3,1,1,1,268,503,2218,0,312,312
3,4,571,11,Santa Clara,450.407906,0.000117,0.052042,"POLYGON ((-121.80597 37.27730, -121.80558 37.2...",4,1,1,1,52,85,18250,1,75,76
4,5,576,11,Santa Clara,423.273023,0.000110,0.043438,"POLYGON ((-121.82788 37.30855, -121.82648 37.3...",5,1,1,1,497,810,18613,13,712,725


## Base Zoning

In [19]:
print(fiona.listlayers(r'M:\Data\GIS layers\UrbanSim_publicMaps\MapForShare\MapForShare.gdb'))
print(fiona.listlayers(r'M:\Data\GIS layers\UrbanSim_publicMaps\UrbanSim Data Review Option B v1d.gdb'))
print(fiona.listlayers(r'M:\Data\GIS layers\UrbanSim_publicMaps\PLU_analysis.gdb'))

['p10_geo_shp']
['p10_boc_opt_b_v1d_geo_yq']
['p10_table', 'p10_boc_v3_geo_tbl_20200311', 'p10_pba50_tbl']


In [20]:
## BASIS BOC data

# p10 geography
p10 = gpd.read_file(r'M:\Data\GIS layers\UrbanSim_publicMaps\MapForShare\MapForShare.gdb\p10_geo_shp.shp')

# p10 zoningmodcast w/ new juris designation
pz10 = pd.read_csv('2020_03_06_zoning_parcels.csv')

# pba50 basezoning PLU

p10_plu50_raw = gpd.read_file(r'M:\Data\GIS layers\UrbanSim_publicMaps\UrbanSim Data Review Option B v1d.gdb', layer='p10_boc_opt_b_v1d_geo_yq')

## merge
p10_z10 = p10.merge(pz10, on = 'PARCEL_ID', how = 'left')
print(p10_z10.shape)
display(p10_z10.head())

(1956208, 24)


,PARCEL_ID,ACRES,COUNTY_ID,ZONE_ID,APN,GEOM_ID,ID,CENTROID,X,Y,...,Unnamed: 0,geom_id,zoning_id,zoning,juris,prop,tablename,nodev_pba40,nodev,juris_id
0,229116.0,3.360520,1.0,725.0,099 029001700,1.030511e+13,14224.0,0101000020D00A00000FDDF39AF3C53C41889F50423032...,-121.795620,37.655379,...,1771381.0,1.030511e+13,60126.0,107 - Urban Low Residential UL2,-9999.0,100.0,plu06,0.0,0.0,livr
1,244166.0,1.294423,1.0,715.0,099B540210200,1.110735e+13,16362.0,0101000020D00A000081F795B4C5E23C419ACAA6DA1667...,-121.713004,37.717277,...,1362648.0,1.110735e+13,11903.0,GP-ULM,99.0,100.0,livermoregeneralplan,0.0,0.0,livr
2,202378.0,14.993605,1.0,820.0,085A643106000,1.103018e+13,16900.0,0101000020D00A0000BA9166219F7A3C411C7B5DC25834...,-122.014199,37.655260,...,307258.0,1.103018e+13,11803.0,LDR,98.0,100.0,hayward_gp_landuse,0.0,0.0,hayw
3,2004420.0,316.247146,97.0,1401.0,141-100-012,6.381678e+12,19308.0,0101000020D00A00001ED46C59D1803B4134422EA6C9E0...,-122.771868,38.727893,...,1737625.0,6.381678e+12,12975.0,LEA240,109.0,100.0,sonomacountygeneralplan,0.0,0.0,uson
4,340332.0,0.621275,1.0,763.0,525 166004800,3.148755e+11,25266.0,0101000020D00A0000AF541F4B8E873C418111D5C36DD5...,-121.974508,37.546277,...,273989.0,3.148755e+11,2511.0,RESM4,5.0,100.0,fremontgeneralplan,0.0,1.0,frem


In [47]:
plu50 = p10_plu50_raw.loc[:,['parcel_id_p10','me','mt', 'mr', 'rb', 'rs', 'ih', 'iw', 'il', 'sc', 'ho', 'of', 'hm', 'ht', 'hs',
                       'max_height','max_dua','max_far','plu_county','plu_jurisdiction','plu_id','plu_code','plu_description','building_types_source','source']]

boc = p10_z10.merge(plu50, left_on = 'PARCEL_ID', right_on = 'parcel_id_p10', how = 'left')

In [48]:
zoning = boc[['PARCEL_ID', 'ACRES', 'COUNTY_ID','juris_id', 'geometry','plu_county','plu_jurisdiction','plu_id','plu_code', 'plu_description', 
                    'max_far', 'max_dua', 'max_height', 'plu_county', 'plu_jurisdiction','building_types_source','source', 
                    'hm', 'ho', 'hs', 'ht', 'ih', 'il', 'iw', 'me', 'mr', 'mt', 'of', 'rb', 'rs', 'sc']] 

In [49]:
juris_name = pd.read_csv(r'M:\Data\GIS layers\UrbanSim_publicMaps\jurisdictions.csv')
juris_name.columns = ['jurisdiction'] + list(juris_name)[1:]
zoning = zoning.merge(juris_name[['jurisdiction','juris']], left_on = 'juris_id', right_on = 'juris',how = 'left')
zoning.drop(columns = ['juris','juris_id'], inplace = True)
display(zoning.head())

,PARCEL_ID,ACRES,COUNTY_ID,geometry,plu_county,plu_jurisdiction,plu_id,plu_code,plu_description,max_far,...,il,iw,me,mr,mt,of,rb,rs,sc,jurisdiction
0,229116.0,3.360520,1.0,"POLYGON ((605993.404 4168316.032, 605993.297 4...",Alameda,Livermore,fc5f982a-40e3-452f-a660-f5f80b7d0b24,PUD-58-93,Planned Development,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,livermore
1,244166.0,1.294423,1.0,"POLYGON ((613491.065 4175262.643, 613490.995 4...",Alameda,Livermore,4d5ddd2e-464a-4861-9c1f-88941a0b5676,RS,Residential,0.35,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,livermore
2,202378.0,14.993605,1.0,"POLYGON ((586745.708 4168287.089, 586746.547 4...",Alameda,Hayward,5a6d3363-b377-4431-8ac8-35aa1593225d,PD,Planned Development,0.00,...,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,hayward
3,2004420.0,316.247146,97.0,"POLYGON ((520226.869 4286274.188, 520222.958 4...",Sonoma,Unincorporated Sonoma,0b967b71-e05b-46db-a689-4d3460b22887,LEA,Land Extensive Agriculture,0.00,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,unincorporated_sonoma
4,340332.0,0.621275,1.0,"POLYGON ((590655.279 4155999.718, 590590.988 4...",Alameda,Fremont,9b689549-939f-4288-aad7-d3584fef122f,P-93-3,Planned District,0.01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fremont


In [50]:
zoning_csv = zoning.drop(columns = ['ACRES','COUNTY_ID','geometry'])
zoning_csv.to_csv('MapForShare/base_zoning_csv_new.csv')

In [51]:
#%%time
zoning.to_file('MapForShare/base_zoning_new.shp')

In [54]:
display(zoning.source.value_counts())
display(zoning.building_types_source.value_counts())

Jessica Setiawan    209255
CCSF_SE             152722
Zoning              109078
General Plan         11415
Name: source, dtype: int64

2010 Aksel Geo-matching                         551168
Ordinance (Joshua Croff)                        460412
Single.*Residential zn_description inference    358744
Ordinance (consultants)                         176356
R1 inference                                    105186
Multi.*Residential zn_description inference      32587
2010                                             24940
Industrial zn_description inference              13685
Office zn_description inference                   4660
Heavy Industrial zn_description inference          917
Warehouse|Logistic zn_description inference        141
Hotel zn_description inference                     125
Name: building_types_source, dtype: int64

## UGB - Urban Growth Boundaries

In [30]:
# Will be added directly from 'https://mtc.maps.arcgis.com/home/item.html?id=cee518ed990947de8d2290416306461f#overview'

## PBA50 Zoningmods

In [15]:
pba50_att_raw = gpd.read_file(r'M:\Data\GIS layers\UrbanSim_publicMaps\PLU_analysis.gdb', layer='p10_pba50_tbl')
pba50_att_raw['pba50chcat_cp'] = pba50_att_raw['pba50chcat']
pba50_att_raw.loc[pba50_att_raw['pba50chcat_cp'].str.contains('plea'),'pba50chcat'] = pba50_att_raw.loc[pba50_att_raw['pba50chcat_cp'].str.contains('plea'),'pba50chcat'].apply(lambda x: x.replace('plea', ''))
pba50_att_raw.loc[pba50_att_raw['pba50chcat_cp'].str.contains('dubl'),'pba50chcat'] = pba50_att_raw.loc[pba50_att_raw['pba50chcat_cp'].str.contains('dubl'),'pba50chcat'].apply(lambda x: x.replace('dubl', ''))
#pba50_att_raw['pba50chcat'].unique()

p10_geo = gpd.read_file(r'M:\Data\GIS layers\UrbanSim_publicMaps\MapForShare\MapForShare.gdb\p10_geo_shp.shp')

In [40]:
pba50_att = pba50_att_raw[['PARCEL_ID', 'COUNTY_ID', 'juris_id', 'gg_id', 'tra_id', 'sesit_id', 'ppa_id', 'exp2020_id', 'pba50chcat', 'exsfd_id', 'chcatwsfd', 'pba50zoningmodcat', 'nodev']]
#pba50_att_tbl = pba50_att.drop(columns = ['COUNTY_ID'])
#pba50_att_tbl.to_csv('MapForShare/pba50_zoningmods.csv')

In [17]:
pba50_att = pba50_att_raw[['PARCEL_ID', 'COUNTY_ID', 'juris_id', 'gg_id', 'tra_id', 'sesit_id', 'ppa_id', 'exp2020_id', 'pba50chcat', 'exsfd_id', 'chcatwsfd', 'pba50zoningmodcat', 'nodev']]
pba50_att['modTemp'] = pba50_att[['juris_id', 'gg_id', 'tra_id', 'sesit_id', 'ppa_id', 'exp2020_id', 'exsfd_id', 'chcatwsfd']].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)
p10 = p10_geo[['PARCEL_ID','geometry']]
zoningmods = pba50_att[['PARCEL_ID','modTemp']].merge(p10, on = 'PARCEL_ID', how = 'right')

C:\Users\ywang\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [18]:
gdf = gpd.GeoDataFrame(zoningmods, geometry='geometry')
gdf.to_file('MapForShare/pba50_zoningmods_20200330.shp')

In [ ]:
print(gdf.shape)
gdf2 = gdf.loc[gdf.geometry.notnull(),:]
print(gdf2.shape)

In [44]:
# dissolve by pba50zoningmodca
gdf2['geometry'] = gdf2.buffer(0.001)
try:
    zoningmod_dis = gdf2.dissolve(by='modTemp',as_index=False)
except:
    print(gdf2.modTemp)

display(zoningmod_dis.head())

C:\Users\ywang\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,modTemp,geometry,PARCEL_ID
0,alam_GG_NA_HRADR_NA_in_NA_None,"MULTIPOLYGON (((566201.479 4176919.045, 566201...",148948.0
1,alam_GG_NA_HRADR_NA_in_sfd_None,"MULTIPOLYGON (((566746.423 4177166.389, 566746...",148108.0
2,alam_GG_NA_HRA_NA_in_NA_None,"MULTIPOLYGON (((567554.216 4176165.250, 567554...",145947.0
3,alam_GG_NA_HRA_NA_in_sfd_None,"MULTIPOLYGON (((567655.312 4176232.388, 567655...",147663.0
4,alam_GG_NA_NA_NA_in_NA_None,"MULTIPOLYGON (((566517.287 4181164.980, 566517...",152599.0


In [45]:
df = pd.DataFrame(zoningmod_dis, copy=True)
df_split = pd.concat([df['modTemp'].str.split('_', expand=True), df['geometry'],df['PARCEL_ID']], axis=1)
display(df_split)

,0,1,2,3,4,5,6,7,8,geometry,PARCEL_ID
0,alam,GG,NA,HRADR,NA,in,NA,None,None,"MULTIPOLYGON (((566201.479 4176919.045, 566201...",148948.0
1,alam,GG,NA,HRADR,NA,in,sfd,None,None,"MULTIPOLYGON (((566746.423 4177166.389, 566746...",148108.0
2,alam,GG,NA,HRA,NA,in,NA,None,None,"MULTIPOLYGON (((567554.216 4176165.250, 567554...",145947.0
3,alam,GG,NA,HRA,NA,in,sfd,None,None,"MULTIPOLYGON (((567655.312 4176232.388, 567655...",147663.0
4,alam,GG,NA,NA,NA,in,NA,None,None,"MULTIPOLYGON (((566517.287 4181164.980, 566517...",152599.0
...,...,...,...,...,...,...,...,...,...,...,...
2583,wood,NA,NA,NA,NA,out,NA,None,None,"POLYGON ((565563.330 4136854.897, 565563.330 4...",1214018.0
2584,wood,NA,NA,NA,NA,out,sfd,None,None,"POLYGON ((565563.135 4136823.811, 565562.600 4...",1214329.0
2585,youn,NA,NA,DR,NA,in,NA,None,None,"MULTIPOLYGON (((553991.839 4248189.753, 553978...",864933.0
2586,youn,NA,NA,DR,NA,in,sfd,None,None,"MULTIPOLYGON (((555983.214 4250414.917, 555983...",864987.0


In [46]:
df_split.columns = ['juris_id', 'gg_id', 'tra_id', 'sesit_id', 'ppa_id', 'exp2020_id', 'exsfd_id', 'chcatwsfd','NA','geometry','PARCEL_ID']
df_split = df_split.merge(juris_name[['jurisdiction','juris']], left_on = 'juris_id', right_on = 'juris',how = 'left')
df_split.drop(columns = ['juris','juris_id'], inplace = True)
df_split['pba50_zoningmods'] = df_split[['gg_id', 'tra_id', 'sesit_id', 'ppa_id', 'exp2020_id', 'exsfd_id']].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)

In [50]:
print(dissolved.shape)
display(dissolved.head())

(2588, 11)

In [48]:
dissolved = gpd.GeoDataFrame(df_split, geometry='geometry')
dissolved.to_file('MapForShare/pba50_zoningmods_diss.shp')